# Multifit

This is an example of a proof of concept of building a single neural network on both image and tabular data.

It doesn't do any better than a linear model with embeddings - but I haven't experimented much with architecture/hyperparameters/etc.

## Read in the data

In [ ]:
from fastai import *
from fastai.vision import *
from fastai.tabular import *
from fastai.callbacks import *

In [ ]:
import fastai
fastai.__version__

In [ ]:
kappa = KappaScore()
kappa.weights = "quadratic"

In [ ]:
from sklearn.metrics import cohen_kappa_score
qwk = partial(cohen_kappa_score, weights="quadratic")

In [ ]:
path = Path('../input/petfinder-adoption-prediction')
work_path = Path('../working')

In [ ]:
train_df = pd.read_csv(path / 'train/train.csv')
test_df = pd.read_csv(path / 'test/test.csv')

In [ ]:
train_df['valid'] = (train_df.RescuerID > 'd')
train_df.valid.mean()

In [ ]:
train_df['photo'] = train_df.PetID.apply(lambda x: f'train_images/{x}-1.jpg')
train_df['has_photo'] = train_df.photo.apply(lambda x: (path / x).exists())
test_df['photo'] = test_df.PetID.apply(lambda x: f'test_images/{x}-1.jpg')
test_df['has_photo'] = test_df.photo.apply(lambda x: (path / x).exists())
(~train_df['has_photo']).sum(), len(train_df)

## Tabular Model

This is the first bucketing I tried; there are probably better options but this worked better for me than feeding these variables a continuous variables into a 2-layer neural net

In [ ]:
train_df['age'] = train_df.Age.apply(lambda x: f'{x} mth' if x < 12 else f'{min(x//12, 6)} yrs')
train_df['quantity'] = train_df.Quantity.apply(lambda x: min(x, 5))
train_df['fee'] = np.log1p(train_df['Fee'])

test_df['age'] = test_df.Age.apply(lambda x: f'{x} mth' if x < 12 else f'{min(x//12, 6)} yrs')
test_df['quantity'] = test_df.Quantity.apply(lambda x: min(x, 5))
test_df['fee'] = np.log1p(test_df['Fee'])

In [ ]:
cat_cols = ['age',
                      'Breed1',
                      'Gender',
                      'Color1',
                      'MaturitySize', 'FurLength',
                      'Vaccinated', 'Dewormed', 'Sterilized', 'Health',
                      'quantity', 'State', 'PhotoAmt']
cont_cols = ['fee']

In [ ]:
procs = [FillMissing, Categorify, Normalize]
data_tab = (TabularList
        .from_df(train_df, cat_names=cat_cols, cont_names=cont_cols, procs=procs)
        .split_from_df(col='valid')
        .label_from_df(cols=['AdoptionSpeed'])
        .add_test(TabularList.from_df(test_df, cat_names=cat_cols, cont_names=cont_cols, procs=procs))
        .databunch(bs=32))

Non-linear model seems to do no better then a linear model (with embeddings)

In [ ]:
tab_learn = tabular_learner(data_tab, layers=[], metrics=[kappa, accuracy])

In [ ]:
tab_learn.fit_one_cycle(4, max_lr=1e-2)

In [ ]:
tab_pred, _ = tab_learn.get_preds(DatasetType.Test)
tab_valid, tab_targ = tab_learn.get_preds(DatasetType.Valid)

In [ ]:
qwk(train_df[train_df.valid].AdoptionSpeed, tab_valid.argmax(1))

## Image Model

In [ ]:
if not Path('../working/train_images').exists():
    os.symlink(path /'train_images', '../working/train_images', target_is_directory=True)
if not Path('../working/test_images').exists():
    os.symlink(path / 'test_images', '../working/test_images', target_is_directory=True)

In [ ]:
transforms = get_transforms()
data_vis = (ImageList
        .from_df(train_df[train_df.has_photo].copy(), work_path, cols=['photo'])
        .split_from_df('valid')
        .label_from_df(cols=['AdoptionSpeed'])
        .transform(transforms, size=224)
        .add_test(ImageList.from_df(test_df[test_df.has_photo].copy(), path, cols=['photo']))
        # Pytorch error with too many workers
        .databunch(bs=32, num_workers=0)
        .normalize(imagenet_stats)
       )

In [ ]:
Path('/tmp/.torch/models/').mkdir(parents=True, exist_ok = True)
shutil.copy('../input/resnet34/resnet34.pth', '/tmp/.torch/models/resnet34-333f7ec4.pth')

In [ ]:
vis_learn = cnn_learner(data_vis, models.resnet34, pretrained=True, metrics=[kappa, accuracy])

In [ ]:
vis_learn.fit_one_cycle(2, max_lr=3e-3)

In [ ]:
vis_learn.unfreeze()
vis_learn.fit_one_cycle(3, max_lr=slice(3e-6, 3e-4))

In [ ]:
vis_learn.save('vis')

In [ ]:
vis_pred, _ = vis_learn.get_preds(DatasetType.Test)
vis_valid, targ = vis_learn.get_preds(DatasetType.Valid)

In [ ]:
valid_mask = train_df.valid & train_df.has_photo

In [ ]:
qwk(train_df.AdoptionSpeed[train_df.valid & train_df.has_photo], vis_valid.argmax(1))

## Combinind Predictions

In [ ]:
vis_mask = train_df[train_df.valid].has_photo
vis_idx = list(np.where(vis_mask))

In [ ]:
qwk(train_df.AdoptionSpeed[train_df.valid & train_df.has_photo], (vis_valid+tab_valid[vis_idx]).argmax(1))

## Combined Model

In [ ]:
tfms = get_transforms()
tfms = [[tfms[0], []], [tfms[0], []]]

In [ ]:
procs = [FillMissing, Categorify, Normalize]
list_tab = (TabularList
            .from_df(train_df[train_df.has_photo],
             cat_names=cat_cols,
             cont_names=cont_cols,
             procs=procs))

In [ ]:
list_vis = (ImageList
            .from_df(train_df[train_df.has_photo], path, cols=['photo']))

 ## Todo: Normalize Images with imagenet stats

In [ ]:
data = (MixedItemList([list_vis, list_tab], path='.', inner_df=train_df[train_df.has_photo])
        .split_from_df('valid')
        .label_from_df(cols=['AdoptionSpeed'])
        .transform(tfms, size=224)
        .databunch(bs=32, num_workers=0))

In [ ]:
emb_szs = data_tab.get_emb_szs({})
emb_szs

In [ ]:
nt = 100

In [ ]:
tab = TabularModel(emb_szs, len(data_tab.cont_names), out_sz=nt, layers=[], bn_final=True)

In [ ]:
class Ridealong(nn.Module):
    """Run m on the first element of list, and bring the others along for the ride"""
    def __init__(self, m, posn=0):
        super().__init__()
        self.m = m
        self.posn = posn
        
    def forward(self, *xs):
        x = xs[self.posn]
        if isinstance(x, Tensor): x = [x]
        x = self.m(*x)
        xs = (*xs[:self.posn], x, *xs[self.posn+1:])
        return xs

In [ ]:
class CombineVisTab(nn.Module):
    def __init__(self, nf, ts, nc):
        super().__init__()
        self.nf = nf
        self.ts = ts
        self.nc = nc
        self.flat = nn.Sequential(AdaptiveConcatPool2d(), Flatten())
        self.layers = nn.Sequential(*bn_drop_lin(nf + ts, nc))
        
    def forward(self, *xs):
        vis, tab = xs
        x = torch.cat([self.flat(vis), tab], dim=1)
        return self.layers(x)
        

In [ ]:
body = create_body(models.resnet34, pretrained=True)
nf = num_features_model(body) * 2

In [ ]:
class MultiSeq(nn.Sequential):
    def forward(self, *input):
        for module in self._modules.values():
            input = module(*input)
        return input

In [ ]:
model = MultiSeq(
    Ridealong(body[:6]),
    Ridealong(body[6:]),
    MultiSeq(
        Ridealong(tab, 1),
        CombineVisTab(nf, nt, data.c)
    )
)
    


In [ ]:
gc.collect()

In [ ]:
learn = Learner(data, model, metrics=[accuracy, kappa])

In [ ]:
learn = learn.split([model[0], model[1]])

In [ ]:
learn.freeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(2, max_lr=slice(1e-3))

In [ ]:
learn.save('l1')

In [ ]:
learn.unfreeze()

Not significantly better than the linear model

In [ ]:
learn.fit_one_cycle(3, max_lr=slice(1e-6, 1e-4))

TODO: Evaluate on Test Data

Results aren't very good so not much point

In [ ]:
#sub_df = pd.DataFrame(data={'PetID': test_df.PetID, 'AdoptionSpeed': pred.argmax(1).numpy()})
#sub_df.to_csv('submission.csv', index=False)